<a href="https://colab.research.google.com/github/tsanoop887-hash/AIF360/blob/main/alphaone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [2]:
class Chess4x4:
    def __init__(self) :
        self.reset()

    def reset(self):
        self.board = np.array([
            [4,2,3,6],
            [1,1,1,1],
            [-1,-1,-1,-1],
            [-4,-2,-3,-6]
        ],dtype=int)
        self.current_player=1
        return self.board

    def in_bounds(self,x,y):
        return 0 <= x < 4 and 0 <= y < 4

    def get_legal_moves(self):
        moves = []
        positions = list(zip(*np.where(self.board*self.current_player>0)))
        for x,y in positions:
            piece = abs(self.board[x,y])
            # Pawn
            if piece==1:
                dx = -1 if self.current_player==1 else 1
                if self.in_bounds(x+dx,y) and self.board[x+dx,y]==0:
                    moves.append(((x,y),(x+dx,y)))
                for dy in [-1,1]:
                    if self.in_bounds(x+dx,y+dy) and self.board[x+dx,y+dy]*self.current_player<0:
                        moves.append(((x,y),(x+dx,y+dy)))
            # Knight
            elif piece==2:
                for dx,dy in [(-2,-1),(-2,1),(-1,-2),(-1,2),(1,-2),(1,2),(2,-1),(2,1)]:
                    nx,ny = x+dx,y+dy
                    if self.in_bounds(nx,ny) and self.board[nx,ny]*self.current_player<=0:
                        moves.append(((x,y),(nx,ny)))
            # Bishop
            elif piece==3:
                for dx,dy in [(-1,-1),(-1,1),(1,-1),(1,1)]:
                    nx,ny = x+dx,y+dy
                    while self.in_bounds(nx,ny):
                        if self.board[nx,ny]*self.current_player>0: break
                        moves.append(((x,y),(nx,ny)))
                        if self.board[nx,ny]*self.current_player<0: break
                        nx,ny = nx+dx,ny+dy
            # Rook
            elif piece==4:
                for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                    nx,ny = x+dx,y+dy
                    while self.in_bounds(nx,ny):
                        if self.board[nx,ny]*self.current_player>0: break
                        moves.append(((x,y),(nx,ny)))
                        if self.board[nx,ny]*self.current_player<0: break
                        nx,ny = nx+dx,ny+dy
            # Queen
            elif piece==5:
                for dx,dy in [(-1,-1),(-1,1),(1,-1),(1,1),(-1,0),(1,0),(0,-1),(0,1)]:
                    nx,ny = x+dx,y+dy
                    while self.in_bounds(nx,ny):
                        if self.board[nx,ny]*self.current_player>0: break
                        moves.append(((x,y),(nx,ny)))
                        if self.board[nx,ny]*self.current_player<0: break
                        nx,ny = nx+dx,ny+dy
            # King
            elif piece==6:
                for dx in [-1,0,1]:
                    for dy in [-1,0,1]:
                        nx,ny = x+dx,y+dy
                        if self.in_bounds(nx,ny) and self.board[nx,ny]*self.current_player<=0 and (dx!=0 or dy!=0):
                            moves.append(((x,y),(nx,ny)))
        return moves

    def make_move(self,move):
        (fx,fy),(tx,ty) = move
        self.board[tx,ty] = self.board[fx,fy]
        self.board[fx,fy] = 0
        self.current_player *= -1

    def is_game_over(self):
        kings = np.where(np.abs(self.board)==6)
        return len(kings[0])<2

    def get_winner(self):
        kings = np.where(np.abs(self.board)==6)
        if len(kings[0])<2:
            return self.current_player*-1
        return 0

In [3]:
class AlphaoneNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 2)
        self.fc1 = nn.Linear(64 * 3 * 3, 128)
        self.policy_head = nn.Linear(128, 16 * 16)
        self.value_head = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        p = torch.softmax(self.policy_head(x), dim=1)
        v = torch.tanh(self.value_head(x))
        return p, v

In [4]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = {}
        self.N = {} # Visit counts
        self.W = {} # Total value
        self.Q = {} # Mean value
        self.P = {} # Prior probability from policy net

def mcts(root, net, sims=100, c_puct=1.0):
    for _ in range(sims):
        node = root
        path = []

        # Selection
        while node.children:
            ucb_max = -float('inf')
            best_move = None
            for move, child in node.children.items():
                # Calculate UCB
                # Add a small value to the denominator to prevent division by zero if N[move] is 0
                u = node.Q[move] + c_puct * node.P.get(move, 0) * np.sqrt(sum(node.N.values()) + 1) / (node.N.get(move, 0) + 1e-8)
                if u > ucb_max:
                    ucb_max = u
                    best_move = move
            # If there are children but no best_move found (e.g., due to all moves having infinite negative UCB, which shouldn't happen with this formula, but as a safeguard)
            if best_move is None:
                # Fallback to selecting a random child if selection fails
                if node.children:
                    best_move = random.choice(list(node.children.keys()))
                else:
                    # If no children, break from selection loop
                    break


            node = node.children[best_move]
            path.append((node, best_move))

        # Check if game is over at the selected node
        if node.state.is_game_over():
            value = node.state.get_winner()
            # Backpropagate the value
            for path_node, move in reversed(path):
                # Ensure move exists in path_node.N before accessing
                if move in path_node.N:
                    path_node.N[move] += 1
                    # Value is from the perspective of the player *after* the move
                    path_node.W[move] += value * path_node.state.current_player
                    path_node.Q[move] = path_node.W[move] / path_node.N[move]
            continue # Go to the next simulation

        # Expansion and Evaluation
        legal_moves = node.state.get_legal_moves()
        if legal_moves:
            board_tensor = torch.tensor(node.state.board, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
            with torch.no_grad():
                p, v = net(board_tensor)
            p = p.squeeze().numpy()

            # Initialize child nodes and their stats for all legal moves
            move_probs = {}
            sum_probs = 0
            for move in legal_moves:
                # Flatten the move to a single index
                from_idx = move[0][0] * 4 + move[0][1]
                to_idx = move[1][0] * 4 + move[1][1]
                flat_move_idx = from_idx * 16 + to_idx # Assuming 16*16 policy head output

                # Get probability from network, handle potential index out of bounds (though 16*16 should cover all 4x4 moves)
                prob = p[flat_move_idx] if flat_move_idx < len(p) else 0.0
                move_probs[move] = prob
                sum_probs += prob

                # Create child node if it doesn't exist and initialize stats
                if move not in node.children:
                    new_state = Chess4x4()
                    new_state.board = np.copy(node.state.board) # Deep copy the board
                    new_state.current_player = node.state.current_player
                    new_state.make_move(move)
                    child_node = Node(new_state, parent=node)
                    node.children[move] = child_node

                    # Initialize child node's stats
                    node.N[move] = 0
                    node.W[move] = 0
                    node.Q[move] = 0
                    # Initialize P with a small value or uniform if sum_probs is 0
                    node.P[move] = prob if sum_probs > 1e-8 else 1.0 / len(legal_moves)


            # Normalize probabilities if sum_probs > 0
            if sum_probs > 1e-8:
                for move in legal_moves:
                    node.P[move] = move_probs[move] / sum_probs
            else:
                 # If sum of probabilities is zero, distribute uniformly among legal moves
                 for move in legal_moves:
                    node.P[move] = 1.0 / len(legal_moves)


            # Backpropagate the value from the neural network (from the perspective of the current player)
            value = v.item()
            for path_node, move in reversed(path):
                # Ensure move exists in path_node.N before accessing
                if move in path_node.N:
                    path_node.N[move] += 1
                    # Value is from the perspective of the player *after* the move
                    path_node.W[move] += value * path_node.state.current_player
                    path_node.Q[move] = path_node.W[move] / path_node.N[move]


        else: # No legal moves, game might be a draw or the player is checkmated
             # In this simplified version, we'll just backpropagate a value of 0 (draw)
             value = 0
             for path_node, move in reversed(path):
                # Ensure move exists in path_node.N before accessing
                if move in path_node.N:
                    path_node.N[move] += 1
                    path_node.W[move] += value * path_node.state.current_player
                    path_node.Q[move] = path_node.W[move] / path_node.N[move]

    # Return normalized visit counts for training policy
    policy = {}
    N_total = sum(root.N.values())
    if N_total > 0:
        for move, n in root.N.items():
            policy[move] = n / N_total
    # If N_total is 0 (e.g., no simulations completed or root had no legal moves),
    # return a uniform policy over legal moves as a fallback.
    elif root.state.get_legal_moves():
         legal_moves = root.state.get_legal_moves()
         for move in legal_moves:
             policy[move] = 1.0 / len(legal_moves)
    # If no legal moves at all from the root, return empty policy
    else:
        policy = {}


    return policy

In [6]:
  while node.children:
            ucb_max = -float('inf')
            best_move = None
            for move,child in node.children.items():
                u = node.Q[move] + c_puct*node.P[move]*np.sqrt(sum(node.N.values())+1)/(1+node.N[move])
                if u>ucb_max:
                    ucb_max = u
                    best_move = move
            node = node.children[best_move]
            path.append((node,best_move))

        # Evaluation
        board_tensor = torch.tensor(node.state.board,dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        with torch.no_grad():
            p,v = net(board_tensor)
        p = p.squeeze().numpy()

        # Expansion
        legal_moves = node.state.get_legal_moves()
        for i,move in enumerate(legal_moves):
            if move not in node.children:
                node.children[move] = Node(state=node.state)
                node.P[move] = p[i] if i<len(p) else 1/len(legal_moves)
                node.N[move] = 0
                node.W[move] = 0
                node.Q[move] = 0

        # Backup
        for parent_node,move in reversed(path):
            parent_node.N[move] += 1
            parent_node.W[move] += v.item()
            parent_node.Q[move] = parent_node.W[move]/parent_node.N[move]

    # Return normalized policy for training
    N_total = sum(root.N.values())
    policy = {move:n/N_total for move,n in root.N.items()}
    return policy

# ==============================
# 4. Self-Play
# ==============================
def play_game(net, sims=100):
    game = Chess4x4()
    states, pis, zs = [],[],[]
    while not game.is_game_over():
        root = Node(game)
        policy = mcts(root, net, sims=sims)
        if policy:
            move = max(policy,key=policy.get)
        else:
            move = random.choice(game.get_legal_moves())
        states.append(game.board.copy())
        pis.append([policy.get(m,0) for m in root.children])
        game.make_move(move)
    winner = game.get_winner()
    zs = [winner]*len(states)
    return states,pis,zs

# ==============================
# 5. Training
# ==============================
def train_network(net, replay_buffer, optimizer, batch_size=64):
    batch = random.sample(replay_buffer,min(batch_size,len(replay_buffer)))
    state_batch = torch.tensor([b[0] for b in batch],dtype=torch.float32).unsqueeze(1)
    policy_batch = torch.tensor([b[1] for b in batch],dtype=torch.float32)
    value_batch = torch.tensor([b[2] for b in batch],dtype=torch.float32).unsqueeze(1)

    optimizer.zero_grad()
    p_pred,v_pred = net(state_batch)
    policy_loss = -(policy_batch*torch.log(p_pred+1e-8)).sum(dim=1).mean()
    value_loss = nn.MSELoss()(v_pred,value_batch)
    loss = policy_loss + value_loss
    loss.backward()
    optimizer.step()
    return loss.item()

# ==============================
# 6. Human vs AI
# ==============================
def human_vs_ai(net, sims=100):
    game = Chess4x4()
    while not game.is_game_over():
        print(game.board)
        if game.current_player==1:
            legal = game.get_legal_moves()
            for i,m in enumerate(legal):
                print(f"{i}: {m}")
            move_idx = int(input(f"Your move (0-{len(legal)-1}): "))
            move = legal[move_idx]
            game.make_move(move)
        else:
            root = Node(game)
            policy = mcts(root, net, sims=sims)
            move = max(policy,key=policy.get)
            print("AI moves:", move)
            game.make_move(move)
    winner = game.get_winner()
    print("Winner:", "Human" if winner==1 else "AI" if winner==-1 else "Draw")

# ==============================
# 7. Main Loop
# ==============================
if __name__=="__main__":
    net = AlphaZeroNet()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    replay_buffer = []

    # Self-play training
    for iteration in range(10):  # increase for stronger AI
        states,pis,zs = play_game(net, sims=50)
        for s,p,z in zip(states,pis,zs):
            replay_buffer.append((s,p,z))
        loss = train_network(net,replay_buffer,optimizer)
        print(f"Iteration {iteration}, Loss: {loss

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)

In [ ]:
# ==============================
# 4. Self-Play
# ==============================
def play_game(net, sims=100):
    game = Chess4x4()
    states, pis, zs = [], [], []
    while not game.is_game_over():
        root = Node(game)
        policy = mcts(root, net, sims=sims)
        if policy:
            # Select move based on visit counts (proportional to policy from MCTS)
            moves, visit_counts = zip(*policy.items())
            # Add a small epsilon to avoid division by zero if all visit counts are zero
            sum_visits = sum(visit_counts) + 1e-8
            move_probabilities = np.array(visit_counts) / sum_visits
            chosen_move_index = np.random.choice(len(moves), p=move_probabilities)
            move = moves[chosen_move_index]

        else:
            # If MCTS returns no legal moves (shouldn't happen if game.get_legal_moves() is correct)
            # Fallback to random move (should ideally not be reached in a correct implementation)
            legal_moves = game.get_legal_moves()
            if legal_moves:
                move = random.choice(legal_moves)
            else:
                break # Should indicate a draw or stalemate if no legal moves

        states.append(game.board.copy())

        # Convert policy dictionary to a list of probabilities corresponding to all possible moves (for fixed policy size)
        # This assumes the policy head in AlphaoneNet outputs probabilities for all possible 16*16 moves
        # We need a consistent ordering of moves to match the policy output
        # A simple approach is to map each (from, to) move to a unique index
        # from_idx = move[0][0] * 4 + move[0][1]
        # to_idx = move[1][0] * 4 + move[1][1]
        # flat_move_idx = from_idx * 16 + to_idx

        # To get the policy vector matching the neural network output, we need to
        # create a vector of size 16*16, where each element corresponds to a
        # possible move (from, to) in flattened index form, and populate it
        # with the probabilities from the MCTS policy for the legal moves.
        # This is complex because the MCTS policy only contains legal moves.

        # A simpler approach for now is to store the policy dictionary directly,
        # or convert it to a format that can be easily used for training.
        # For training, we need a policy vector where illegal moves have zero probability.

        # Let's create a policy vector of size 16*16 where legal moves have their probabilities
        # and illegal moves have 0.
        policy_vector = np.zeros(16 * 16)
        total_visits = sum(policy.values()) + 1e-8 # Add epsilon for stability
        for m, visits in policy.items():
            from_idx = m[0][0] * 4 + m[0][1]
            to_idx = m[1][0] * 4 + m[1][1]
            flat_move_idx = from_idx * 16 + to_idx
            policy_vector[flat_move_idx] = visits / total_visits

        pis.append(policy_vector)

        game.make_move(move)

    winner = game.get_winner()
    zs = [winner] * len(states)
    return states, pis, zs

# ==============================
# 5. Training
# ==============================
def train_network(net, replay_buffer, optimizer, batch_size=64):
    if not replay_buffer:
        return 0.0  # Return 0 loss if replay buffer is empty

    batch = random.sample(replay_buffer, min(batch_size, len(replay_buffer)))

    state_batch = torch.tensor([b[0] for b in batch], dtype=torch.float32).unsqueeze(1)
    # Ensure policy_batch has the correct shape (batch_size, 16*16)
    policy_batch = torch.tensor([b[1] for b in batch], dtype=torch.float32)
    value_batch = torch.tensor([b[2] for b in batch], dtype=torch.float32).unsqueeze(1)

    optimizer.zero_grad()
    p_pred, v_pred = net(state_batch)

    # Ensure p_pred and policy_batch have compatible shapes for loss calculation
    # If p_pred is (batch_size, 256) and policy_batch is (batch_size, 256), this is fine.
    policy_loss = -(policy_batch * torch.log(p_pred + 1e-8)).sum(dim=1).mean()
    value_loss = nn.MSELoss()(v_pred, value_batch)
    loss = policy_loss + value_loss
    loss.backward()
    optimizer.step()
    return loss.item()

# ==============================
# 6. Human vs AI
# ==============================
def human_vs_ai(net, sims=100):
    game = Chess4x4()
    while not game.is_game_over():
        print(game.board)
        if game.current_player == 1:
            legal = game.get_legal_moves()
            if not legal:
                print("No legal moves for Human. Game ends.")
                break
            for i, m in enumerate(legal):
                print(f"{i}: {m}")
            try:
                move_idx = int(input(f"Your move (0-{len(legal)-1}): "))
                if 0 <= move_idx < len(legal):
                    move = legal[move_idx]
                    game.make_move(move)
                else:
                    print("Invalid move index. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number.")
        else:
            root = Node(game)
            policy = mcts(root, net, sims=sims)
            if policy:
                # Select the move with the highest visit count
                move = max(policy, key=policy.get)
                print("AI moves:", move)
                game.make_move(move)
            else:
                print("AI has no legal moves. Game ends.")
                break # Should indicate a draw or stalemate

    winner = game.get_winner()
    if winner == 1:
        print("Winner: Human")
    elif winner == -1:
        print("Winner: AI")
    else:
        print("Draw")

# ==============================
# 7. Main Loop
# ==============================
if __name__ == "__main__":
    net = AlphaoneNet()  # Use AlphaoneNet as defined in the other cell
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    replay_buffer = []

    # Self-play training
    for iteration in range(10):  # increase for stronger AI
        print(f"Starting iteration {iteration}...")
        states, pis, zs = play_game(net, sims=50)
        print(f"Game finished in iteration {iteration}. Collected {len(states)} states.")
        for s, p, z in zip(states, pis, zs):
            replay_buffer.append((s, p, z))
        print(f"Replay buffer size: {len(replay_buffer)}")
        if replay_buffer:
            loss = train_network(net, replay_buffer, optimizer)
            print(f"Iteration {iteration}, Loss: {loss}")
        else:
            print(f"Iteration {iteration}, No data in replay buffer to train.")


    # Example of playing against the AI (optional)
    # print("\nStarting Human vs AI game...")
    # human_vs_ai(net, sims=100)

In [7]:
game = Chess4x4()
print("Initial board state:")
print(game.board)
print("Current player:", game.current_player)

Initial board state:
[[ 4  2  3  6]
 [ 1  1  1  1]
 [-1 -1 -1 -1]
 [-4 -2 -3 -6]]
Current player: 1
